In [1]:
!aws s3 rm s3://itversitydata/ghactivity --recursive

In [2]:
import boto3

dynamodb = boto3.resource('dynamodb')
table = dynamodb.Table('jobs')
table.delete_item(Key={'job_id': 'ghactivity_ingest'})
item = {
    'job_id': 'ghactivity_ingest',
    'job_description': 'Ingest data from gmail to s3',
    'is_active': 'Y',
    'baseline_days': 3
}
table.put_item(Item=item)

{'ResponseMetadata': {'RequestId': '18RH7VV7C0LAT85JJ4442F7A2NVV4KQNSO5AEMVJF66Q9ASUAAJG',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'server': 'Server',
   'date': 'Sat, 04 Jun 2022 00:44:48 GMT',
   'content-type': 'application/x-amz-json-1.0',
   'content-length': '2',
   'connection': 'keep-alive',
   'x-amzn-requestid': '18RH7VV7C0LAT85JJ4442F7A2NVV4KQNSO5AEMVJF66Q9ASUAAJG',
   'x-amz-crc32': '2745614147'},
  'RetryAttempts': 0}}

In [3]:
import requests

In [4]:
next_file = '2022-06-01-0.json.gz'

In [5]:
res = requests.get(f'https://data.gharchive.org/{next_file}')

In [6]:
s3_client = boto3.client('s3')

In [7]:
upload_res = s3_client.put_object(
   Bucket='itversitydata',
   Key=f'ghactivity/{next_file}',
   Body=res.content
)


In [8]:

print(upload_res)

{'ResponseMetadata': {'RequestId': 'FSKC89G1FM3Y22PX', 'HostId': '6zSKY1f+TfIO/s/B+fyDsKo4ElVr9epgAiD6cd12oG/2Y9n1kIh7YJF1CV9I8K/1Jpw5WMmVS4c=', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amz-id-2': '6zSKY1f+TfIO/s/B+fyDsKo4ElVr9epgAiD6cd12oG/2Y9n1kIh7YJF1CV9I8K/1Jpw5WMmVS4c=', 'x-amz-request-id': 'FSKC89G1FM3Y22PX', 'date': 'Sat, 04 Jun 2022 00:45:01 GMT', 'etag': '"fcb8d11ef628f8482af9d915b131e485"', 'server': 'AmazonS3', 'content-length': '0'}, 'RetryAttempts': 0}, 'ETag': '"fcb8d11ef628f8482af9d915b131e485"'}


In [9]:
!aws s3 ls s3://itversitydata/ghactivity/

2022-06-04 06:15:01   98882105 2022-06-01-0.json.gz
